## Import libraries and dependencies


In [2]:
import data_prep as dp
import data_analysis as da


import datetime as dt

import pandas as pd
import yfinance as yf
import quantstats as qs
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import plotly.graph_objs as go                    #create candlestick charts  
import plotly.express as px                       #create Violin plots, pie charts, box plot, scatter plots,line graph, area

import plotly.io as pio
pio.renderers.default ='colab'                   #Initializing plotly

qs.extend_pandas()

ModuleNotFoundError: No module named 'data_analysis'

## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [ ]:
portfolio_choice = 0

while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio, type 2\n"
        )
                              )
    # Print error message if type(portfolio_choice) != int                        
    except ValueError:
        print('Error: Invalid response.\n')
        
    # Print error message if portfolio choice not 1 or 2
    if portfolio_choice not in [1, 2]:
        print('Error: Invalid response.\n')

# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        ticker_list = pd.DataFrame(columns=ticker_list).add_suffix('-USD').columns.tolist()
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")
    

In [ ]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)



In [ ]:
# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]

# Print portfolio data for visual confirmation
if portfolio_choice == 1:    
    # Calculate portfolio value of each cryptocurrency held
    for ticker in existing_portfolio:
        existing_portfolio[ticker].append(
            {'value': portfolio_df[f"{ticker}-USD"].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = 0
    for ticker, units in existing_portfolio.items():
        value = existing_portfolio[ticker][1]['value']
        print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
        portfolio_value += value
    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       
    
elif portfolio_choice == 2:
    print(f"--------------------------")                      
    print(f"Hypothetical Portfolio:")
    print(f"{[ticker.replace('-USD', '') for ticker in ticker_list]}")                   
    print(f"Investment amount:")
    print(f"${portfolio_value:.2f}\n")

print(
    f"NOTE:\n"
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")



## Data Analyses

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [ ]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)


# Calculate proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)



In [ ]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        


In [ ]:
display(ratios_df)

In [ ]:
display(portfolio_df)

# Visulalizations

In [ ]:
btc=portfolio_df['BTC-USD']

In [ ]:
eth=portfolio_df['ETH-USD']

## Pairplots

In [ ]:
sns.pairplot(btc)

In [ ]:
sns.pairplot(eth)

In [ ]:
sns.pairplot(ratios_df)

## Line graph

In [ ]:
portfolio_df.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

In [ ]:
btc.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

In [ ]:
eth.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
ax2 = ax1.twinx()
rspine = ax2.spines['right']
rspine.set_position(('axes', 1.15))
ax2.set_frame_on(True)
ax2.patch.set_visible(False)
fig.subplots_adjust(right=0.7)
btc.plot(ax=ax1, style='b-')
eth.plot(ax=ax1, style='r-', secondary_y=True)

In [ ]:
ratios_df.plot(grid=True, figsize=(15, 10)).axhline(y = 1, color = "black", lw = 2)

## Heatmap

In [ ]:
# df2=portfolio_df.corr()
# sns.heatmap(df2,cmap="Greens", annot=True)

## Bar Graph 

In [ ]:
ratios_df.plot(kind='bar',figsize= (20,10))

## Boxplot visuliazation to find outliers

In [ ]:
ax = sns.boxplot(data=portfolio_df, orient="h")

In [ ]:
ax = sns.boxplot(data=btc, orient="h")

In [ ]:
et = sns.boxplot(data=eth, orient="h")

In [ ]:
rd=sns.boxplot(data=ratios_df, orient="h")

## Scatter Plot

In [ ]:
px.scatter(btc,hover_data = ['high','low','open','close'], height= 400)

In [ ]:
px.scatter(eth, hover_data = ['high','low','open','close'], height= 400)

## Area Chart

In [ ]:
px.area(btc, hover_data = ['high','low','open','close'], height= 600)

In [ ]:
px.area(eth, hover_data = ['high','low','open','close'], height= 600)

## Candlestick figure

In [ ]:
fig = go.Figure(data = [go.Candlestick(x=portfolio_df.index,
                       open=btc.open,
                       high=btc.high,
                       low=btc.low,
                       close=btc.close)
                                       ]) 

In [ ]:
fig.show()

In [ ]:
fig = go.Figure(data = [go.Candlestick(x=portfolio_df.index,
                       open=eth.open,
                       high=eth.high,
                       low=eth.low,
                       close=eth.close)
                                       ]) 

In [ ]:
fig.show()

In [ ]:
# import pandas as pd
# import numpy as np
# import panel as pn
# from panel.interact import interact
# from panel import widgets

In [ ]:
# pn.extension()

In [ ]:
# Import hvplot.pandas after pn.extension
# This avoids plotly initialization failure
# import hvplot.pandas

In [ ]:
# portfolio_df.reset_index()

In [ ]:
# Define function to choose a year
# def choose_Date(Date):
#     return Date

In [ ]:
# Execute interact function
# interact(choose_Date, Date)

## Visualizations of Tableau

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1641984633922' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BarCharts1_16419846320250&#47;Multiple_Bar_Chart' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641984633922');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1641984513275' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='BarCharts_16419845132190&#47;Bar_Charts2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641984513275');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Visualization for 1 Hour Quote USD percent Change (for years 2013-2020)

In [ ]:
%%HTML
<div class='tableauPlaceholder' id='viz1641983762340' style='position: relative'><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='path' value='views&#47;1H_16419837362410&#47;1H?:language=en-GB&amp;:embed=true&amp;publish=yes' /> <param name='toolbar' value='yes' /><param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-GB' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1641983762340');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>